# MIP for KEP

In [4]:
# Enumerate cycles of length < L in graph G

function enum_smallcycles(G, L)
    paires = 1:nv(G)
    smallcycles = []
    function rec_enum(p,c,L)
        for q in neighbors(G,p)
            if q == c[1]
                push!(smallcycles,c)
            elseif q < c[1] 
                continue
            elseif q in c
                continue
            elseif length(c) <= L
                ccopy = copy(c)
                push!(ccopy,q)
                rec_enum(q,ccopy,L)
            end 
        end
    end
    for p in paires
        c = [p]
        rec_enum(p,copy(c),L)
    end
    
    # weight for each cycle
    wcycles = []
    for c in smallcycles 
        push!(wcycles,get_prop(G,Edge(c[length(c)],c[1]),:weight)+sum(get_prop(G, Edge(c[i],c[i+1]), :weight) for i in 1:(length(c)-1)))
    end
        
    return smallcycles,wcycles
end

enum_smallcycles (generic function with 1 method)

In [1]:
# solve MIP for graph G with maximal cercle length L

function solve_MIP(G, L)
    C,Wc = enum_smallcycles(G, L)
    
    KEP = Model(Gurobi.Optimizer)

    @variable(KEP,z[c in 1:length(C)],Bin)
    
    # Constraint: each pair is chosen at most 1 time
    @constraint(KEP,choosecerc[i in 1:nv(G)],sum(z[c] for c in 1:length(C) if i in C[c]) <= 1)

    # Objective: Maximise total score
    @objective(KEP,Max, sum(Wc[c] *z[c] for c in 1:length(C)))
    
    optimize!(KEP)

    return JuMP.objective_value(KEP),JuMP.value.(z)
end

LoadError: LoadError: UndefVarError: @variable not defined
in expression starting at In[1]:8